In [1]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import PublicFormat, Encoding, load_der_public_key
from cryptography.hazmat.primitives.asymmetric import ec, utils
from cryptography.exceptions import InvalidSignature

In [2]:
# generate txid function
def generate_txid(sender_hash : bytes,recipient_hash : bytes, sender_public_key : bytes, amount : int, fee : int, nonce : int, signature : bytes):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(sender_hash)
    digest.update(recipient_hash)
    digest.update(sender_public_key)
    digest.update((amount).to_bytes(8, byteorder = 'little', signed = False))
    digest.update((fee).to_bytes(8, byteorder = 'little', signed = False))
    digest.update((nonce).to_bytes(8, byteorder = 'little', signed = False))
    digest.update(signature)
    return digest.finalize()
# generate signature function
def generate_signature(recipient_hash : bytes, amount : int, fee : int, nonce : int):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(recipient_hash)
    digest.update((amount).to_bytes(8, byteorder = 'little', signed = False))
    digest.update((fee).to_bytes(8, byteorder = 'little', signed = False))
    digest.update((nonce).to_bytes(8, byteorder = 'little', signed = False))
    return digest.finalize()
# calculate sha1 hash function
def calculate_sha1_hash(public_key : bytes):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(public_key)
    return digest.finalize()
# generate public key from private key
def private_key_to_public_key(private_key):
    return private_key.public_key().public_bytes(encoding=Encoding.DER, format=PublicFormat.SubjectPublicKeyInfo)

#constructor
class Transaction:
    
    def __init__ (self,sender_hash : bytes,recipient_hash : bytes ,sender_public_key : bytes ,amount : int, fee : int ,
                  nonce : int ,signature : bytes,txid : bytes):
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid  
    
    def verify(self,sender_balance,sender_previous_nonce):
        assert len(self.sender_hash) == 20, \
            f"Sender's hash {(self.sender_hash)} is not 20 bytes long"
        assert len(self.recipient_hash) == 20, \
            f"Recipients's hash {(self.recipient_hash)} is not 20 bytes long"
        assert self.amount <= sender_balance, \
            f"Sender's balance of {sender_balance} does not cover amount {(self.amount)}"
        assert self.fee <= self.amount, \
            f"Fee {(self.fee)} exceeds amount {(self.amount)}"
        assert self.nonce == sender_previous_nonce + 1 or self.nonce == -1 + 1, \
            f"Sender's nonce of {sender_previous_nonce} does not match expected {(self.nonce)}"
        assert self.txid == generate_txid(self.sender_hash,self.recipient_hash, self.sender_public_key,\
                                           self.amount, self.fee, self.nonce, self.signature) , \
            f"Transaction ID is not valid !"
        try:
            load_der_public_key(self.sender_public_key).verify(self.signature, \
                            generate_signature(self.recipient_hash,self.amount,self.fee,self.nonce), \
                            ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        except InvalidSignature :
            
            raise InvalidSignature("Signature is not valid !")

# create transaction function
def create_signed_transaction(sender_private_key, recipient_hash, amount, fee, nonce):
    signature_hash = sender_private_key.sign(generate_signature(recipient_hash,amount,fee,nonce), ec.ECDSA(utils.Prehashed(hashes.SHA256())))
    sender_public_key = private_key_to_public_key(sender_private_key)
    sender_hash = calculate_sha1_hash(sender_public_key)
    txid = generate_txid(sender_hash,recipient_hash,sender_public_key,amount,fee,nonce,signature_hash)
    return Transaction(sender_hash, recipient_hash, sender_public_key, amount, fee, nonce, signature_hash, txid )

### Test 1 
Generate a private key using ec.generate_private_key(ec.SECP256K1) . Call
create_signed_transaction to make a test transaction. Check that the
transaction.verify call succeeds (when provided with a sender_balance which is
sufficiently high and sender_previous_nonce = transaction.nonce - 1 ).

In [3]:
# Question 1
# generate sender and recipient private key
sender_sk_1 = ec.generate_private_key(
    ec.SECP256K1())
recipient_sk_1 = ec.generate_private_key(
    ec.SECP256K1())
# calculate recipient hash hashing the recipient public key
recipient_hash_1 = calculate_sha1_hash(private_key_to_public_key(recipient_sk_1))
# create signed transaction
transaction_1 = create_signed_transaction(sender_sk_1, recipient_hash_1, 4, 1, 1)
# verify transaction
transaction_1.verify(sender_balance=20,sender_previous_nonce=0)

### Test 2
Generate a valid transaction, check that modifying any of the fields causes
transaction.verify to raise an exception due to an invalid txid .

In [4]:
# Question 2
# generate sender and recipient private key
sender_sk_2 = ec.generate_private_key(
    ec.SECP256K1())
recipient_sk_2 = ec.generate_private_key(
    ec.SECP256K1())
# calculate recipient hash hashing the recipient public key
recipient_hash_2 = calculate_sha1_hash(private_key_to_public_key(recipient_sk_2))
# create signed transaction
transaction_2 = create_signed_transaction(sender_sk_2, recipient_hash_2, 4, 1, 1)
# verify transaction
transaction_2.verify(sender_balance=10,sender_previous_nonce=0)

In [5]:
# modify fee
transaction_2.fee = 2
# verify transaction
transaction_2.verify(sender_balance=10,sender_previous_nonce=0)

AssertionError: Transaction ID is not valid !

### Test 3
Generate a valid transaction, change the amount field, regenerate the txid so it is valid
again. Check that transaction.verify raises an exception due to an invalid signature.

In [6]:
# Question 3
# generate sender and recipient private key
sender_sk_3 = ec.generate_private_key(
    ec.SECP256K1())
recipient_sk_3 = ec.generate_private_key(
    ec.SECP256K1())
# calculate recipient hash hashing the recipient public key
recipient_hash_3 = calculate_sha1_hash(private_key_to_public_key(recipient_sk_3))
# create signed transaction
transaction_3 = create_signed_transaction(sender_sk_3, recipient_hash_3, 4, 1, 1)
# verify transaction
transaction_3.verify(sender_balance=10,sender_previous_nonce=0)

In [7]:
# modify amount
transaction_3.amount = 10
# rigenerate txid
transaction_3.txid = generate_txid(transaction_3.sender_hash,transaction_3.recipient_hash, 
                                         transaction_3.sender_public_key,transaction_3.amount, \
                                        transaction_3.fee, transaction_3.nonce, transaction_3.signature)
# verify transaction
transaction_3.verify(sender_balance=20,sender_previous_nonce=0)

InvalidSignature: Signature is not valid !

### Test 4
Generate a valid transaction, check that transaction.verify raises an exception if either
the sender_balance is too low or sender_previous_nonce is incorrect

In [8]:
# Question 4
# generate sender and recipient private key
sender_sk_4 = ec.generate_private_key(
    ec.SECP256K1())
recipient_sk_4 = ec.generate_private_key(
    ec.SECP256K1())
# calculate recipient hash hashing the recipient public key
recipient_hash_4 = calculate_sha1_hash(private_key_to_public_key(recipient_sk_4))
# create signed transaction
transaction_4 = create_signed_transaction(sender_sk_4, recipient_hash_4, 30, 1, 1)
# verify transaction
transaction_4.verify(sender_balance=20,sender_previous_nonce=0)

AssertionError: Sender's balance of 20 does not cover amount 30

### Test 5
Generate two private keys, A and B . Use A to generate a valid transaction. Replace the
signature with a signature created using B . Regenerate the txid and confirm that
transaction.verify fails with an invalid signature.


In [9]:
# Question 5
# generate 2 sender private keys and recipient private key
sk_A = ec.generate_private_key(
    ec.SECP256K1())
sk_B = ec.generate_private_key(
    ec.SECP256K1())
sk_recipient = ec.generate_private_key(
    ec.SECP256K1())
# calculate recipient hash hashing the recipient public key
recipient_hash_A = calculate_sha1_hash(private_key_to_public_key(sk_recipient))
# create signed transaction
transaction_5 = create_signed_transaction(sk_A, recipient_hash_A, 30, 1, 1)
# replace signature
transaction_5.signature = sk_B.sign(generate_signature(transaction_5.recipient_hash,\
transaction_5.amount,transaction_5.fee,transaction_5.nonce), ec.ECDSA(utils.Prehashed(hashes.SHA256())))
# rigenerate txid
transaction_5.txid = generate_txid(transaction_5.sender_hash,transaction_5.recipient_hash,\
transaction_5.sender_public_key,transaction_5.amount,transaction_5.fee, transaction_5.nonce,transaction_5.signature)
# verify transaction
transaction_5.verify(sender_balance = 40, sender_previous_nonce = 0)

InvalidSignature: Signature is not valid !

### Test 6
Check that the following transaction verifies successfully (when using sender_balance = 20 ,
sender_previous_nonce = 4 )


In [10]:
transaction_6 =Transaction(
bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d"),
bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca"),
bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
"03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
"b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
"31373afe8d1c860a9ac47d8e2659b74d437435b0" +
"5f2c55bf3f033ac1"),
10,
2,
5,
bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
"2f173378cf78cf79c7978a2337fbad141d022100" +
"ec27704d4d604f839f99e62c02e65bf60cc93ae1"
"735c1ccf29fd31bd3c5a40ed"),
bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" +
"6c2b936e1e788c5c87657bc3"))

transaction_6.verify(20,4)